In [1]:
!pip install torch torchvision timm matplotlib scikit-learn opencv-python


In [2]:
from google.colab import files
uploaded = files.upload()


Saving archive (6).zip to archive (6).zip


In [4]:
import zipfile

zip_path = "archive (6).zip"  # use the uploaded filename exactly
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/brain_tumor_dataset")


In [5]:
import os

print("Files inside dataset folder:")
print(os.listdir("/content/brain_tumor_dataset"))


Files inside dataset folder:
['Training', 'Testing']


In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # resize for EfficientNet/Swin
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]) # normalize
])

train_data = datasets.ImageFolder("/content/brain_tumor_dataset/Training", transform=transform)
test_data  = datasets.ImageFolder("/content/brain_tumor_dataset/Testing", transform=transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=16, shuffle=False)

print("Classes:", train_data.classes)
print("Training samples:", len(train_data))
print("Testing samples:", len(test_data))


Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
Training samples: 5712
Testing samples: 1311


In [8]:
import torch
import torch.nn as nn
import timm  # for pretrained Swin Transformer

class HybridModel(nn.Module):
    def __init__(self, num_classes=4):
        super(HybridModel, self).__init__()

        # --- CNN Block for low-level feature extraction ---
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        # --- Swin Transformer (pretrained) ---
        self.swin = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True)
        in_features = self.swin.head.in_features
        self.swin.head = nn.Identity()  # remove final layer

        # --- Fusion of CNN + Swin ---
        self.fc = nn.Sequential(
            nn.Linear(in_features + 64*56*56, 512),  # combine features
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # CNN path
        cnn_out = self.cnn(x)
        cnn_out = cnn_out.view(cnn_out.size(0), -1)  # flatten

        # Swin path
        swin_out = self.swin(x)

        # Concatenate CNN + Swin features
        combined = torch.cat((cnn_out, swin_out), dim=1)

        return self.fc(combined)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=4).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [11]:
def forward(self, x):
    # CNN features
    cnn_out = self.cnn(x)
    cnn_out = self.cnn_fc(cnn_out)

    # Swin features
    swin_out = self.swin(x)
    swin_out = swin_out.view(swin_out.size(0), -1)  # flatten (B, C*H*W)
    swin_out = self.swin_fc(swin_out)

    # Concatenate
    combined = torch.cat((cnn_out, swin_out), dim=1)

    return self.fc(combined)


In [12]:
import torch
import torch.nn as nn
import torchvision.models as models
from transformers import SwinModel

class HybridModel(nn.Module):
    def __init__(self, num_classes=4):
        super(HybridModel, self).__init__()

        # CNN backbone (EfficientNet or ResNet)
        cnn_base = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(cnn_base.children())[:-1])  # remove final FC
        self.cnn_fc = nn.Linear(cnn_base.fc.in_features, 256)

        # Swin Transformer backbone
        self.swin = SwinModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
        self.swin_fc = nn.Linear(self.swin.config.hidden_size, 256)

        # Final classification head
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # CNN forward
        cnn_out = self.cnn(x)                      # (B, 512, 1, 1)
        cnn_out = cnn_out.view(cnn_out.size(0), -1) # flatten -> (B, 512)
        cnn_out = self.cnn_fc(cnn_out)              # (B, 256)

        # Swin forward
        swin_out = self.swin(x).pooler_output       # already (B, hidden_size)
        swin_out = self.swin_fc(swin_out)           # (B, 256)

        # Combine
        combined = torch.cat((cnn_out, swin_out), dim=1)  # (B, 512)

        return self.fc(combined)


In [13]:
model = HybridModel(num_classes=4).to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 179MB/s]


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

In [16]:
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
from transformers import SwinModel

class HybridModel(nn.Module):
    def __init__(self, num_classes=4):
        super(HybridModel, self).__init__()

        # CNN backbone (ResNet18)
        self.cnn = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.cnn.fc = nn.Identity()  # remove last FC layer

        # Swin Transformer backbone
        self.swin = SwinModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

        # Final classifier (CNN features + Swin features)
        self.fc = nn.Linear(512 + self.swin.config.hidden_size, num_classes)

    def forward(self, x):
        # CNN branch
        cnn_out = self.cnn(x)  # (batch, 512)

        # Swin branch
        swin_out = self.swin(x).pooler_output  # (batch, hidden_size)

        # Concatenate features
        combined = torch.cat((cnn_out, swin_out), dim=1)

        return self.fc(combined)


In [17]:
model = HybridModel(num_classes=4).to(device)


In [18]:
import torch.optim as optim

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training function
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation after each epoch
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}, Val Acc: {acc:.2f}%")

    print("✅ Training Finished!")

# Run training
train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5)


Epoch [1/5], Loss: 86.9479, Val Acc: 97.10%
Epoch [2/5], Loss: 26.2560, Val Acc: 98.47%
Epoch [3/5], Loss: 15.9442, Val Acc: 98.17%
Epoch [4/5], Loss: 11.6569, Val Acc: 98.02%
Epoch [5/5], Loss: 6.8706, Val Acc: 98.86%
✅ Training Finished!


In [19]:
torch.save(model.state_dict(), "brain_tumor_hybrid.pth")


In [33]:
import os

test_glioma_path = "/content/brain_tumor_dataset/Testing/glioma"
print("Available files:", os.listdir(test_glioma_path))


Available files: ['Te-gl_0210.jpg', 'Te-gl_0168.jpg', 'Te-gl_0251.jpg', 'Te-gl_0047.jpg', 'Te-gl_0293.jpg', 'Te-gl_0271.jpg', 'Te-gl_0093.jpg', 'Te-gl_0160.jpg', 'Te-gl_0162.jpg', 'Te-gl_0087.jpg', 'Te-gl_0295.jpg', 'Te-gl_0265.jpg', 'Te-gl_0149.jpg', 'Te-gl_0267.jpg', 'Te-gl_0036.jpg', 'Te-gl_0268.jpg', 'Te-gl_0186.jpg', 'Te-gl_0010.jpg', 'Te-gl_0233.jpg', 'Te-gl_0064.jpg', 'Te-gl_0294.jpg', 'Te-gl_0100.jpg', 'Te-gl_0263.jpg', 'Te-gl_0058.jpg', 'Te-gl_0280.jpg', 'Te-gl_0184.jpg', 'Te-gl_0045.jpg', 'Te-gl_0259.jpg', 'Te-gl_0236.jpg', 'Te-gl_0252.jpg', 'Te-gl_0077.jpg', 'Te-gl_0201.jpg', 'Te-gl_0094.jpg', 'Te-gl_0204.jpg', 'Te-gl_0178.jpg', 'Te-gl_0083.jpg', 'Te-gl_0034.jpg', 'Te-gl_0260.jpg', 'Te-gl_0145.jpg', 'Te-gl_0030.jpg', 'Te-gl_0220.jpg', 'Te-gl_0146.jpg', 'Te-gl_0275.jpg', 'Te-gl_0216.jpg', 'Te-gl_0049.jpg', 'Te-gl_0241.jpg', 'Te-gl_0164.jpg', 'Te-gl_0237.jpg', 'Te-gl_0208.jpg', 'Te-gl_0020.jpg', 'Te-gl_0104.jpg', 'Te-gl_0085.jpg', 'Te-gl_0080.jpg', 'Te-gl_0032.jpg', 'Te-gl_015

In [20]:
from google.colab import files

files.download("brain_tumor_hybrid.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_v2_s

# Example Hybrid model skeleton (adjust according to your training code)
class HybridBrainTumorModel(nn.Module):
    def __init__(self, num_classes=2):
        super(HybridBrainTumorModel, self).__init__()
        self.cnn = efficientnet_v2_s(weights=None)
        self.cnn.classifier = nn.Identity()  # remove original classifier
        self.transformer = nn.TransformerEncoderLayer(d_model=1280, nhead=8)
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.cnn(x)  # CNN feature extraction
        # Here, you may need to reshape / pass through Swin transformer
        x = x.flatten(1)  # flatten for FC
        x = self.fc(x)
        return x


In [26]:
!cat /content/brain_tumor_model.py


cat: /content/brain_tumor_model.py: No such file or directory


In [28]:
!ls /content/


'archive (6).zip'   brain_tumor_dataset   brain_tumor_hybrid.pth   sample_data


In [29]:
import os

dataset_path = "/content/brain_tumor_dataset"  # path to your extracted dataset
print(os.listdir(dataset_path))


['Training', 'Testing']


In [30]:
import os

dataset_path = "/content/brain_tumor_dataset"
print("Classes in Training folder:", os.listdir(os.path.join(dataset_path, "Training")))
print("Classes in Testing folder:", os.listdir(os.path.join(dataset_path, "Testing")))


Classes in Training folder: ['glioma', 'notumor', 'pituitary', 'meningioma']
Classes in Testing folder: ['glioma', 'notumor', 'pituitary', 'meningioma']


In [36]:
import os
from PIL import Image
import torch
from torchvision import transforms

# 1️⃣ Device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# 2️⃣ Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# 3️⃣ Get one file per class automatically
dataset_path = "/content/brain_tumor_dataset/Testing"
classes = ['glioma', 'notumor', 'pituitary', 'meningioma']
test_images = []

for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    files = os.listdir(cls_path)
    if files:
        test_images.append(os.path.join(cls_path, files[0]))  # pick the first file

# 4️⃣ Predict
for img_path in test_images:
    image = Image.open(img_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor)
        pred_class = torch.argmax(output, dim=1).item()
    print(f"{img_path.split('/')[-1]} -> Predicted class: {classes[pred_class]}")


Te-gl_0210.jpg -> Predicted class: notumor
Te-no_0112.jpg -> Predicted class: notumor
Te-pi_0216.jpg -> Predicted class: notumor
Te-me_0253.jpg -> Predicted class: notumor


In [37]:
torch.save(model.state_dict(), "/content/brain_tumor_hybrid_final.pth")


In [38]:
import os
from PIL import Image
from torchvision import transforms
import torch

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# Paths
dataset_path = "/content/brain_tumor_dataset/Testing"
classes = ['glioma', 'notumor', 'pituitary', 'meningioma']

# Predict
for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    files = os.listdir(cls_path)
    if files:
        img_path = os.path.join(cls_path, files[0])  # first image
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            pred_class = torch.argmax(output, dim=1).item()
        print(f"{img_path.split('/')[-1]} -> Predicted class: {classes[pred_class]}")


Te-gl_0210.jpg -> Predicted class: notumor
Te-no_0112.jpg -> Predicted class: notumor
Te-pi_0216.jpg -> Predicted class: notumor
Te-me_0253.jpg -> Predicted class: notumor


In [39]:
import os
from PIL import Image
import torch
from torchvision import transforms

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# Classes and dataset path
dataset_path = "/content/brain_tumor_dataset/Testing"
classes = ['glioma', 'notumor', 'pituitary', 'meningioma']

# Predict first image of each class
for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    files = os.listdir(cls_path)
    if files:
        img_path = os.path.join(cls_path, files[0])
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            pred_class = torch.argmax(output, dim=1).item()
        print(f"{img_path.split('/')[-1]} -> Predicted class: {classes[pred_class]}")


Te-gl_0210.jpg -> Predicted class: notumor
Te-no_0112.jpg -> Predicted class: notumor
Te-pi_0216.jpg -> Predicted class: notumor
Te-me_0253.jpg -> Predicted class: notumor


In [40]:
# Predict first 3 images per class
for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    files = os.listdir(cls_path)[:3]  # pick first 3 images
    for f in files:
        img_path = os.path.join(cls_path, f)
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            pred_class = torch.argmax(output, dim=1).item()
        print(f"{img_path.split('/')[-1]} -> Predicted class: {classes[pred_class]}")


Te-gl_0210.jpg -> Predicted class: notumor
Te-gl_0168.jpg -> Predicted class: notumor
Te-gl_0251.jpg -> Predicted class: notumor
Te-no_0112.jpg -> Predicted class: notumor
Te-no_0397.jpg -> Predicted class: notumor
Te-no_0183.jpg -> Predicted class: notumor
Te-pi_0216.jpg -> Predicted class: notumor
Te-pi_0107.jpg -> Predicted class: notumor
Te-pi_0113.jpg -> Predicted class: notumor
Te-me_0253.jpg -> Predicted class: notumor
Te-me_0190.jpg -> Predicted class: notumor
Te-me_0258.jpg -> Predicted class: notumor


In [41]:
torch.save(model.state_dict(), "/content/brain_tumor_hybrid_final.pth")


In [42]:
import csv

results = [
    ("Te-gl_0210.jpg", "notumor"),
    ("Te-gl_0168.jpg", "notumor"),
    # add all predictions here or generate dynamically from your loop
]

with open("/content/predictions.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Predicted Class"])
    writer.writerows(results)


In [43]:
import os
import csv
from PIL import Image
from torchvision import transforms
import torch

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

dataset_path = "/content/brain_tumor_dataset/Testing"
classes = ['glioma', 'notumor', 'pituitary', 'meningioma']

results = []

for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    files = os.listdir(cls_path)[:3]  # first 3 images
    for f in files:
        img_path = os.path.join(cls_path, f)
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            pred_class = torch.argmax(output, dim=1).item()
        results.append([f, classes[pred_class]])

# Save to CSV
with open("/content/predictions.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Predicted Class"])
    writer.writerows(results)

print("✅ Model and predictions saved! Check /content/")


✅ Model and predictions saved! Check /content/
